# SageMaker Serverless Inference
## HuggingFace Text Classification example

Amazon SageMaker Serverless Inference is a purpose-built inference option that makes it easy for you to deploy and scale ML models. Serverless Inference is ideal for workloads which have idle periods between traffic spurts and can tolerate cold starts. Serverless endpoints automatically launch compute resources and scale them in and out depending on traffic, eliminating the need to choose instance types or manage scaling policies. This takes away the undifferentiated heavy lifting of selecting and managing servers. Serverless Inference integrates with AWS Lambda to offer you high availability, built-in fault tolerance and automatic scaling.

Serverless Inference is a great choice for customers that have intermittent or unpredictable prediction traffic. For example, a document processing service used to extract and analyze data on a periodic basis. Customers that choose Serverless Inference should make sure that their workloads can tolerate cold starts. A cold start can occur when your endpoint doesn’t receive traffic for a period of time. It can also occur when your concurrent requests exceed the current request usage. The cold start time will depend on your model size, how long it takes to download, and your container startup time.

## Introduction

Text Classification can be used to solve various use-cases like sentiment analysis, spam detection, hashtag prediction etc. 


This notebook demonstrates the use of the [HuggingFace `transformers` library](https://huggingface.co/transformers/) together with a custom Amazon sagemaker-sdk extension to fine-tune a pre-trained transformer on multi class text classification. In particular, the pre-trained model will be fine-tuned using the [`20 newsgroups dataset`](http://qwone.com/~jason/20Newsgroups/). To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on.

<b>Notebook Setting</b>
- <b>SageMaker Classic Notebook Instance</b>: `ml.m5.xlarge` Notebook Instance & `conda_pytorch_p36 Kernel`
- <b>SageMaker Studio</b>: `Python 3 (PyTorch 1.6 Python 3.6 CPU Optimized)`
- <b>Regions Available</b>: SageMaker Serverless Inference is currently available in the following regions: US East (Northern Virginia), US East (Ohio), US West (Oregon), EU (Ireland), Asia Pacific (Tokyo) and Asia Pacific (Sydney)

## Table of Contents
- Setup
- Data Preprocessing
- Model Training
- Deployment
    - Endpoint Configuration (Adjust for Serverless)
    - Serverless Endpoint Creation
    - Endpoint Invocation
- Cleanup
- Conclusion

# Development Environment and Permissions 

## Setup

If you run this notebook in SageMaker Studio, you need to make sure `ipywidgets` is installed and restart the kernel, so please uncomment the code in the next cell, and run it.

In [2]:
%%capture
import IPython
import sys

!{sys.executable} -m pip install ipywidgets
IPython.Application.instance().kernel.do_shutdown(True)  # has to restart kernel so changes are used

Let's install the required packages from HuggingFace and SageMaker

In [1]:
import sys

!{sys.executable} -m pip install "scikit_learn>=1.1.2" "sagemaker>=2.110.1" "transformers>=4.21.1" "datasets>=2.2.0" "nltk>=3.4.4"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 33.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 KB 7.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 34.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 KB 6.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 KB 10.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 41.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 KB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 KB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.1/262.1 KB 4.3 MB/s eta 0:00:0000:01
    

In [26]:
!pip install scikit-learn --upgrade

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Make sure SageMaker version is >= 2.75.1

In [3]:
import sagemaker

print(sagemaker.__version__)

2.111.0


In [4]:
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

s3_prefix = "huggingface_serverless/20_newsgroups"

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::415275363822:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole
sagemaker bucket: sagemaker-us-east-1-415275363822
sagemaker session region: us-east-1


### Data Preparation

Now we'll download a dataset from the web on which we want to train the text classification model.

In this example, let us train the text classification model on the [`20 newsgroups dataset`](http://qwone.com/~jason/20Newsgroups/). The `20 newsgroups dataset` consists of 20000 messages taken from 20 Usenet newsgroups.

In [5]:
import os
import shutil

data_dir = "20_newsgroups_bulk"
if os.path.exists(data_dir):  # cleanup existing data folder
    shutil.rmtree(data_dir)

In [6]:
!aws s3 cp s3://sagemaker-sample-files/datasets/text/20_newsgroups/20_newsgroups_bulk.tar.gz .

download: s3://sagemaker-sample-files/datasets/text/20_newsgroups/20_newsgroups_bulk.tar.gz to ./20_newsgroups_bulk.tar.gz


In [8]:
!tar xzf 20_newsgroups_bulk.tar.gz
!ls 20_newsgroups_bulk

tar: 20_newsgroups_bulk/talk.politics.misc: Cannot change ownership to uid 500, gid 500: Operation not permitted
tar: 20_newsgroups_bulk/rec.autos: Cannot change ownership to uid 500, gid 500: Operation not permitted
tar: 20_newsgroups_bulk/rec.sport.baseball: Cannot change ownership to uid 500, gid 500: Operation not permitted
tar: 20_newsgroups_bulk/alt.atheism: Cannot change ownership to uid 500, gid 500: Operation not permitted
tar: 20_newsgroups_bulk/sci.space: Cannot change ownership to uid 500, gid 500: Operation not permitted
tar: 20_newsgroups_bulk/comp.sys.mac.hardware: Cannot change ownership to uid 500, gid 500: Operation not permitted
tar: 20_newsgroups_bulk/comp.windows.x: Cannot change ownership to uid 500, gid 500: Operation not permitted
tar: 20_newsgroups_bulk/comp.graphics: Cannot change ownership to uid 500, gid 500: Operation not permitted
tar: 20_newsgroups_bulk/rec.sport.hockey: Cannot change ownership to uid 500, gid 500: Operation not permitted
tar: 20_newsgrou

In [9]:
file_list = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
print("Number of files:", len(file_list))

Number of files: 20


In [10]:
import pandas as pd

documents_count = 0
for file in file_list:
    df = pd.read_csv(file, header=None, names=["text"])
    documents_count = documents_count + df.shape[0]
print("Number of documents:", documents_count)

Number of documents: 19997


Let's inspect the dataset files and analyze the categories.

In [11]:
categories_list = [f.split("/")[1] for f in file_list]
categories_list

['sci.electronics',
 'rec.sport.baseball',
 'rec.autos',
 'talk.politics.guns',
 'rec.motorcycles',
 'alt.atheism',
 'misc.forsale',
 'talk.politics.mideast',
 'comp.windows.x',
 'comp.graphics',
 'rec.sport.hockey',
 'sci.space',
 'talk.religion.misc',
 'sci.med',
 'talk.politics.misc',
 'comp.sys.mac.hardware',
 'comp.sys.ibm.pc.hardware',
 'sci.crypt',
 'soc.religion.christian',
 'comp.os.ms-windows.misc']

We can see that the dataset consists of 20 topics, each in different file.

Let us inspect the dataset to get some understanding about how the data and the label is provided in the dataset. 

In [12]:
df = pd.read_csv("./20_newsgroups_bulk/rec.motorcycles", header=None, names=["text"])
df

,text
0,Newsgroups: rec.motorcycles\nPath: cantaloupe....
1,Newsgroups: rec.motorcycles\nPath: cantaloupe....
2,Newsgroups: rec.motorcycles\nPath: cantaloupe....
3,Newsgroups: rec.motorcycles\nPath: cantaloupe....
4,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...
...,...
995,Path: cantaloupe.srv.cs.cmu.edu!rochester!corn...
996,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...
997,Newsgroups: rec.motorcycles\nPath: cantaloupe....
998,Newsgroups: rec.motorcycles\nPath: cantaloupe....


In [13]:
df["text"][0]

'Newsgroups: rec.motorcycles\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!agate!linus!linus.mitre.org!mbunix.mitre.org!cookson\nFrom: cookson@mbunix.mitre.org (Cookson)\nSubject: Volvo Attack!\nMessage-ID: <1993Apr21.143403.4644@linus.mitre.org>\nSender: news@linus.mitre.org (News Service)\nNntp-Posting-Host: mbunix.mitre.org\nOrganization: The MITRE Corp., Bedford, Ma.\nDate: Wed, 21 Apr 1993 14:34:03 GMT\nLines: 22\n\nI was privelged enough to experience my first Volvo attack this weekend.\n\nI was last in a line of traffic that was about 6 vehicles long, riding\ndown Rt. 40 in Groton Ma.  At the side of the road, sitting off on the\nshoulder was the killer Volvo in question.  No brake lights, no turn signal,\nnothing.  We were doing about 40 mph and I was following the cage in front\nof me about 2.5-3 sec. back.  Well, as said cage passes the Volvo, the\nBrain Dead Idiot (tm) behind the wheel decides that she 

In [14]:
df = pd.read_csv("./20_newsgroups_bulk/comp.sys.mac.hardware", header=None, names=["text"])
df

,text
0,Newsgroups: comp.sys.mac.hardware\nPath: canta...
1,Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....
2,Newsgroups: comp.sys.mac.hardware\nPath: canta...
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...
4,Newsgroups: comp.sys.mac.hardware\nPath: canta...
...,...
995,Newsgroups: comp.sys.mac.hardware\nPath: canta...
996,Xref: cantaloupe.srv.cs.cmu.edu comp.sys.ibm.p...
997,Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv....
998,Newsgroups: comp.sys.mac.hardware\nPath: canta...


In [15]:
df["text"][0]

'Newsgroups: comp.sys.mac.hardware\nPath: cantaloupe.srv.cs.cmu.edu!rochester!udel!gatech!howland.reston.ans.net!agate!headwall.Stanford.EDU!nntp.Stanford.EDU!cmwand\nFrom: cmwand@leland.Stanford.EDU (Christopher Wand)\nSubject: Re: Syquest 150 ???\nMessage-ID: <1993Apr20.043629.21237@leland.Stanford.EDU>\nSender: news@leland.Stanford.EDU (Mr News)\nOrganization: DSG, Stanford University, CA 94305, USA\nReferences: <93759@hydra.gatech.EDU>\nDistribution: usa\nDate: Tue, 20 Apr 93 04:36:29 GMT\nLines: 30\n\nIn article <93759@hydra.gatech.EDU> gt8798a@prism.gatech.EDU (Anthony S. Kim) writes:\n>I remember someone mention about a 150meg syquest.  Has anyone else\n>heard anything about this?  I\'d be interested in the cost per megabyte and the\n>approximate cost of the drive itself and how they compare to the Bernoulli 150.\n\nI think you must be talking about the Syquest 105 (code named Mesa I believe).\nIt is a 3.5" Winchester technology drive pretty much like the other Syquest\ndrives i

As we can see from the above, there is a single file for each class in the dataset. Each record is just a plain text paragraphs with header, body, footer and quotes. We will need to process them into a suitable data format.

## Data Preprocessing
We need to preprocess the dataset to remove the header, footer, quotes, leading/trailing whitespace, extra spaces, tabs, and HTML tags/markups. 

Download the `nltk` tokenizer and other libraries

In [16]:
import nltk
from nltk.tokenize import word_tokenize
import re
import string

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
import sklearn
print(sklearn.__version__)

1.1.2


In [18]:
from sklearn.datasets._twenty_newsgroups import (
    strip_newsgroup_header,
    strip_newsgroup_quoting,
    strip_newsgroup_footer,
)

This following function will remove the header, footer and quotes (of earlier messages in each text).

In [19]:
def strip_newsgroup_item(item):
    item = strip_newsgroup_header(item)
    item = strip_newsgroup_quoting(item)
    item = strip_newsgroup_footer(item)
    return item

The following function will take care of removing leading/trailing whitespace, extra spaces, tabs, and HTML tags/markups.

In [20]:
def process_text(texts):
    final_text_list = []
    for text in texts:

        # Check if the sentence is a missing value
        if isinstance(text, str) == False:
            text = ""

        filtered_sentence = []

        # Lowercase
        text = text.lower()

        # Remove leading/trailing whitespace, extra space, tabs, and HTML tags/markups
        text = text.strip()
        text = re.sub("\[.*?\]", "", text)
        text = re.sub("https?://\S+|www\.\S+", "", text)
        text = re.sub("<.*?>+", "", text)
        text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
        text = re.sub("\n", "", text)
        text = re.sub("\w*\d\w*", "", text)

        for w in word_tokenize(text):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric
            if not w.isnumeric():
                filtered_sentence.append(w)
        final_string = " ".join(filtered_sentence)  # final string of cleaned words

        final_text_list.append(final_string)

    return final_text_list

Now we will read each of the `20_newsgroups` dataset files, call `strip_newsgroup_item` and `process_text` functions we defined earlier, and then aggregate all data into one dataframe.

In [24]:
all_categories_df = pd.DataFrame()

for file in file_list:
    print(f"Processing {file}")
    label = file.split("/")[1]
    df = pd.read_csv(file, header=None, names=["text"])
    df["text"] = df["text"].apply(strip_newsgroup_item)
    df["text"] = process_text(df["text"].tolist())
    df["label"] = label
    all_categories_df = all_categories_df.append(df, ignore_index=True)
    #all_categories_df = all_categories_df.concat(df, ignore_index=True)

Processing 20_newsgroups_bulk/sci.electronics


<ipython-input-24-a02e212c97eb>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_categories_df = all_categories_df.append(df, ignore_index=True)


Processing 20_newsgroups_bulk/rec.sport.baseball


<ipython-input-24-a02e212c97eb>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_categories_df = all_categories_df.append(df, ignore_index=True)


Processing 20_newsgroups_bulk/rec.autos


...

<ipython-input-24-a02e212c97eb>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_categories_df = all_categories_df.append(df, ignore_index=True)


Processing 20_newsgroups_bulk/soc.religion.christian


<ipython-input-24-a02e212c97eb>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_categories_df = all_categories_df.append(df, ignore_index=True)


Processing 20_newsgroups_bulk/comp.os.ms-windows.misc


<ipython-input-24-a02e212c97eb>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_categories_df = all_categories_df.append(df, ignore_index=True)


Let's inspect how many categories there are in our dataset.

In [25]:
all_categories_df["label"].value_counts()

sci.electronics             1000
rec.sport.baseball          1000
sci.crypt                   1000
comp.sys.ibm.pc.hardware    1000
comp.sys.mac.hardware       1000
talk.politics.misc          1000
sci.med                     1000
talk.religion.misc          1000
sci.space                   1000
rec.sport.hockey            1000
comp.graphics               1000
comp.windows.x              1000
talk.politics.mideast       1000
misc.forsale                1000
alt.atheism                 1000
rec.motorcycles             1000
talk.politics.guns          1000
rec.autos                   1000
comp.os.ms-windows.misc     1000
soc.religion.christian       997
Name: label, dtype: int64

In our dataset there are 20 categories which is too much, so we will combine the sub-categories.

In [26]:
# replace to politics
all_categories_df["label"].replace(
    {
        "talk.politics.misc": "politics",
        "talk.politics.guns": "politics",
        "talk.politics.mideast": "politics",
    },
    inplace=True,
)

# replace to recreational
all_categories_df["label"].replace(
    {
        "rec.sport.hockey": "recreational",
        "rec.sport.baseball": "recreational",
        "rec.autos": "recreational",
        "rec.motorcycles": "recreational",
    },
    inplace=True,
)

# replace to religion
all_categories_df["label"].replace(
    {
        "soc.religion.christian": "religion",
        "talk.religion.misc": "religion",
        "alt.atheism": "religion",
    },
    inplace=True,
)

# replace to computer
all_categories_df["label"].replace(
    {
        "comp.windows.x": "computer",
        "comp.sys.ibm.pc.hardware": "computer",
        "comp.os.ms-windows.misc": "computer",
        "comp.graphics": "computer",
        "comp.sys.mac.hardware": "computer",
    },
    inplace=True,
)
# replace to sales
all_categories_df["label"].replace({"misc.forsale": "sales"}, inplace=True)

# replace to science
all_categories_df["label"].replace(
    {
        "sci.crypt": "science",
        "sci.electronics": "science",
        "sci.med": "science",
        "sci.space": "science",
    },
    inplace=True,
)

Now we are left with 6 categories, which is much better.

In [27]:
all_categories_df["label"].value_counts()

computer        5000
science         4000
recreational    4000
politics        3000
religion        2997
sales           1000
Name: label, dtype: int64

Let's calculate number of words for each row.

In [28]:
all_categories_df["word_count"] = all_categories_df["text"].apply(lambda x: len(str(x).split()))
all_categories_df.head()

,text,label,word_count
0,almost sold but the potential buyer has an opp...,science,125
1,try finding a friend that has caller id then g...,science,82
2,i am not a battery expert but from recent read...,science,129
3,i understand that the new gps boxes now have a...,science,41
4,laser holes are or were used to prevent someon...,science,83


Let's get basic statistics about the dataset.

In [29]:
all_categories_df["word_count"].describe()

count    19997.000000
mean       159.346102
std        434.479067
min          0.000000
25%         37.000000
50%         74.000000
75%        148.000000
max      11351.000000
Name: word_count, dtype: float64

We can see that the mean value is around 159 words. However, there are outliers, such as a text with 11351 words. This can make it harder for the model to result in good performance. We will take care to drop those rows.

Let's drop empty rows first.

In [30]:
no_text = all_categories_df[all_categories_df["word_count"] == 0]
print(len(no_text))

# drop these rows
all_categories_df.drop(no_text.index, inplace=True)

90


Let's drop the rows that are longer than 256 words, as it is a length close to the mean value of the word count. This is done to make it easy for the model to train without outliers. 

In [31]:
long_text = all_categories_df[all_categories_df["word_count"] > 256]
print(len(long_text))

# drop these rows
all_categories_df.drop(long_text.index, inplace=True)

2409


In [32]:
all_categories_df["label"].value_counts()

computer        4659
recreational    3675
science         3506
politics        2370
religion        2349
sales            939
Name: label, dtype: int64

Let's get basic statistics about the dataset after our outliers fixes.

In [33]:
all_categories_df["word_count"].describe()

count    17498.000000
mean        79.797348
std         59.636188
min          1.000000
25%         33.000000
50%         64.000000
75%        113.000000
max        256.000000
Name: word_count, dtype: float64

This looks much more balanced.

Now we drop the `word_count` columns as we will not need it anymore.

In [34]:
all_categories_df.drop(columns="word_count", axis=1, inplace=True)

In [35]:
all_categories_df

,text,label
0,almost sold but the potential buyer has an opp...,science
1,try finding a friend that has caller id then g...,science
2,i am not a battery expert but from recent read...,science
3,i understand that the new gps boxes now have a...,science
4,laser holes are or were used to prevent someon...,science
...,...,...
19992,windows nt or wnt can also be derived by the n...,computer
19993,ive got a probelm with printing envelops on my...,computer
19994,i would like to find a windows driver for a vi...,computer
19995,i have an eisa machine and i just do not under...,computer


Let's convert categorical label to integer number, in order to prepare the dataset for training.

In [36]:
categories = all_categories_df["label"].unique().tolist()
categories

['science', 'recreational', 'politics', 'religion', 'sales', 'computer']

In [37]:
categories.index("recreational")

1

In [38]:
all_categories_df["label"] = all_categories_df["label"].apply(lambda x: categories.index(x))

In [39]:
all_categories_df["label"].value_counts()

5    4659
1    3675
0    3506
2    2370
3    2349
4     939
Name: label, dtype: int64

We partition the dataset into 80% training and 20% validation set and save to `csv` files.

In [40]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(all_categories_df, test_size=0.2)

In [41]:
train_df.to_csv("train.csv", index=None)

In [42]:
test_df.to_csv("test.csv", index=None)

Let's inspect the label distribution in the training dataset

In [43]:
train_df["label"].value_counts()

5    3713
1    2969
0    2800
2    1887
3    1854
4     775
Name: label, dtype: int64

Let's inspect the label distribution in the test dataset

In [44]:
test_df["label"].value_counts()

5    946
1    706
0    706
3    495
2    483
4    164
Name: label, dtype: int64

## Tokenization 

A tokenizer is in charge of preparing the inputs for a model. The library contains tokenizers for all the models. Most of the tokenizers are available in two flavors: a full python implementation and a “Fast” implementation based on the Rust library [tokenizers](https://github.com/huggingface/tokenizers). The “Fast” implementations allows:

 - A significant speed-up in particular when doing batched tokenization.
 - Additional methods to map between the original string (character and words) and the token space (e.g. getting the index of the token comprising a given character or the span of characters corresponding to a given token). 

In [46]:
from datasets import load_dataset
from transformers import AutoTokenizer

# tokenizer used in preprocessing
tokenizer_name = "distilbert-base-uncased"

In [47]:
# download tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

### Load train and test datasets

Let's create a [Dataset](https://huggingface.co/docs/datasets/loading_datasets.html) from our local `csv` files for training and test we saved earlier.

In [48]:
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

Using custom data configuration default-ca8ab14bb88b658d


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ca8ab14bb88b658d/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [49]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 13998
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3500
    })
})

In [53]:
# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

In [50]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

### Tokenize train and test datasets

Let's tokenize the train dataset

In [54]:
train_dataset = dataset["train"].map(tokenize, batched=True)

  0%|          | 0/14 [00:00<?, ?ba/s]

Let's tokenize the test dataset

In [55]:
test_dataset = dataset["test"].map(tokenize, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

### Set format for PyTorch

In [56]:
train_dataset = train_dataset.rename_column("label", "labels")
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

## Uploading data to `sagemaker_session_bucket`

After we processed the datasets, we are going to upload it to S3.

In [57]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()

# save train_dataset to s3
training_input_path = f"s3://{sess.default_bucket()}/{s3_prefix}/train"
train_dataset.save_to_disk(training_input_path, fs=s3)

# save test_dataset to s3
test_input_path = f"s3://{sess.default_bucket()}/{s3_prefix}/test"
test_dataset.save_to_disk(test_input_path, fs=s3)

print(training_input_path)
print(test_input_path)

s3://sagemaker-us-east-1-415275363822/huggingface_serverless/20_newsgroups/train
s3://sagemaker-us-east-1-415275363822/huggingface_serverless/20_newsgroups/test


## Training the HuggingFace model for supervised text classification

In order to create a sagemaker training job we need a `HuggingFace` Estimator. The Estimator handles end-to-end Amazon SageMaker training and deployment tasks. In an Estimator we define, which fine-tuning script should be used as `entry_point`, which `instance_type` should be used, which `hyperparameters` are passed in .....



```python
huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./code',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            volume_size=256,
                            role=role,
                            transformers_version='4.6',
                            pytorch_version='1.7',
                            py_version='py36',
                            hyperparameters = {'epochs': 1,
                                               'model_name':'distilbert-base-uncased',
                                               'num_labels': 6
                                              })
```

When we create a SageMaker training job, SageMaker takes care of starting and managing all the required ec2 instances for us with the `huggingface` container, uploads the provided fine-tuning script `train.py` and downloads the data from our `sagemaker_session_bucket` into the container at `/opt/ml/input/data`. Then, it starts the training job by running. 

```python
/opt/conda/bin/python train.py --epochs 1 --model_name distilbert-base-uncased --num_labels 6
```

The `hyperparameters` you define in the `HuggingFace` estimator are passed in as named arguments. 

SageMaker is providing useful properties about the training environment through various environment variables, including the following:

* `SM_MODEL_DIR`: A string that represents the path where the training job writes the model artifacts to. After training, artifacts in this directory are uploaded to S3 for model hosting.

* `SM_NUM_GPUS`: An integer representing the number of GPUs available to the host.

* `SM_CHANNEL_XXXX:` A string that represents the path to the directory that contains the input data for the specified channel. For example, if you specify two input channels in the HuggingFace estimator’s fit call, named `train` and `test`, the environment variables `SM_CHANNEL_TRAIN` and `SM_CHANNEL_TEST` are set.


To run your training job locally you can define `instance_type='local'` or `instance_type='local-gpu'` for `gpu` usage. _Note: this does not work within SageMaker Studio_


We create a metric_definition dictionary that contains regex-based definitions that will be used to parse the job logs and extract metrics

In [58]:
metric_definitions = [
    {"Name": "loss", "Regex": "'loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "learning_rate", "Regex": "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_loss", "Regex": "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_accuracy", "Regex": "'eval_accuracy': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_f1", "Regex": "'eval_f1': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_precision", "Regex": "'eval_precision': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_recall", "Regex": "'eval_recall': ([0-9]+(.|e\-)[0-9]+),?"},
    {"Name": "eval_runtime", "Regex": "'eval_runtime': ([0-9]+(.|e\-)[0-9]+),?"},
    {
        "Name": "eval_samples_per_second",
        "Regex": "'eval_samples_per_second': ([0-9]+(.|e\-)[0-9]+),?",
    },
    {"Name": "epoch", "Regex": "'epoch': ([0-9]+(.|e\-)[0-9]+),?"},
]

## Creating an Estimator and start a training job

In [59]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters = {"epochs": 3, "model_name": "distilbert-base-uncased", "num_labels": 6}

Now, let's define the SageMaker `HuggingFace` estimator with resource configurations and hyperparameters to train Text Classification on `20 newsgroups` dataset, running on a `p3.2xlarge` instance.

In [60]:
huggingface_estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./",
    instance_type="ml.p3.2xlarge",
    instance_count=1,
    volume_size=256,
    role=role,
    transformers_version="4.10",
    pytorch_version="1.9",
    py_version="py38",
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
)

In [61]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({"train": training_input_path, "test": test_input_path})

ValueError: No file named "train.py" was found in directory "./".

## Deployment

### Serverless Configuration

#### Memory size - `memory_size_in_mb`
Your serverless endpoint has a minimum RAM size of <b>1024 MB (1 GB)</b>, and the maximum RAM size you can choose is 6144 MB (6 GB). The memory sizes you can select are <b>1024 MB</b>, <b>2048 MB</b>, <b>3072 MB</b>, <b>4096 MB</b>, <b>5120 MB</b>, or <b>6144 MB. Serverless Inference auto-assigns compute resources proportional to the memory you select. If you select a larger memory size, your container has access to more `vCPUs`. Select your endpoint’s memory size according to your model size. Generally, the memory size should be at least as large as your model size. You may need to benchmark in order to select the right memory selection for your model based on your latency SLAs. The memory size increments have different pricing; see the Amazon SageMaker pricing page for more information.

#### Concurrent invocations - `max_concurrency`
   
Serverless Inference manages predefined scaling policies and quotas for the capacity of your endpoint. Serverless endpoints have a quota for how many concurrent invocations can be processed at the same time. If the endpoint is invoked before it finishes processing the first request, then it handles the second request concurrently. You can set the maximum concurrency for a <b>single endpoint up to 50</b>, and the total number of serverless endpoint variants you can host in a Region is 50. The total concurrency you can share between all serverless endpoints per Region in your account is 200. The maximum concurrency for an individual endpoint prevents that endpoint from taking up all the invocations allowed for your account, and any endpoint invocations beyond the maximum are throttled.

In [1]:
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=2,
)

### HuggingFace Inference Image `URI`

In order to deploy the SageMaker Endpoint with Serverless configuration, we will need to supply the HuggingFace Inference Image URI.

In [64]:
image_uri = sagemaker.image_uris.retrieve(
    framework="huggingface",
    base_framework_version="pytorch1.9",
    region=sess.boto_region_name,
    version="4.10",
    py_version="py38",
    instance_type="ml.m5.large",
    image_scope="inference",
)
image_uri

'763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-inference:1.9-transformers4.10-cpu-py38-ubuntu20.04'

### Serverless Endpoint Creation
Now that we have a `ServerlessInferenceConfig`, we can create a serverless endpoint and deploy our model to it.

In [ ]:
%%time

predictor = huggingface_estimator.deploy(
    serverless_inference_config=serverless_config, image_uri=image_uri
)

------------

## Endpoint Invocation

Using few samples, you can now invoke the SageMaker endpoint to get predictions.

In [ ]:
def predict_sentence(sentence):
    result = predictor.predict({"inputs": sentence})
    index = int(result[0]["label"].split("LABEL_")[1])
    print(categories[index])

In [ ]:
sentences = [
    "The modem is an internal AT/(E)ISA 8-bit card (just a little longer than a half-card).",
    "In the cage I usually wave to bikers.  They usually don't wave back.  My wife thinks it's strange but I don't care.",
    "Voyager has the unusual luck to be on a stable trajectory out of the solar system.",
]

# using the same processing logic that we used during data preparation for training
processed_sentences = process_text(sentences)

for sentence in processed_sentences:
    predict_sentence(sentence)

### Invoke a deployed endpoint

In [45]:
!aws sagemaker list-endpoints

{
    "Endpoints": [
        {
            "EndpointName": "deepar-electricity-demo-new-features-2022-08-05-21-30-57-463",
            "EndpointArn": "arn:aws:sagemaker:us-east-1:415275363822:endpoint/deepar-electricity-demo-new-features-2022-08-05-21-30-57-463",
            "CreationTime": 1659735058.258,
            "LastModifiedTime": 1659735209.12,
            "EndpointStatus": "InService"
        },
        {
            "EndpointName": "deepar-electricity-demo-2022-08-05-20-48-23-534",
            "EndpointArn": "arn:aws:sagemaker:us-east-1:415275363822:endpoint/deepar-electricity-demo-2022-08-05-20-48-23-534",
            "CreationTime": 1659732504.326,
            "LastModifiedTime": 1659732645.499,
            "EndpointStatus": "InService"
        },
        {
            "EndpointName": "demand-forecast-deepar-2022-08-05-20-25-48-159",
            "EndpointArn": "arn:aws:sagemaker:us-east-1:415275363822:endpoint/demand-forecast-deepar-2022-08-05-20-25-48-159",
            "Cr

In [21]:
from transformers import pipeline
generator = pipeline('text-generation', model = 'gpt2')

In [56]:
Payloads = generator("Elon Musk's Twitter acquisition", max_length = 80, num_return_sequences=200)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [54]:
Payloads

[{'generated_text': "Elon Musk's Twitter acquisition looks to be another example with Tesla. Musk has said that Musk will help the company establish its own distribution center network in California, which will help ensure that all cars sold within that region are sold at the same prices on the rest of the U.S.' roads.\n\nWhile the details of the move were left to Tesla employees, Bloomberg notes that Musk's team"},
 {'generated_text': 'Elon Musk\'s Twitter acquisition of Tesla seems a bit more like a marketing ploy than something to be taken seriously by the media.\n\nNow, I don\'t think it\'s because Musk is a "great guy," as everyone is being falsely accused of, or because he\'s a narcissist; he really has the personality — and his energy and enthusiasm and determination to become the greatest engineer in history'},
 {'generated_text': "Elon Musk's Twitter acquisition is one of the biggest, most interesting and interesting business deals in the internet age. This also has several in

In [57]:
import json
import boto3

client = boto3.client('runtime.sagemaker')
payload = {"inputs": "The modem is an internal AT/(E)ISA 8-bit card (just a little longer than a half-card)."}
for i in range(len(Payloads)):
    Payloads[i]['inputs'] = Payloads[i]['generated_text']
    del Payloads[i]['generated_text']
    response = client.invoke_endpoint(EndpointName='huggingface-pytorch-training-2022-10-05-23-57-02-171', 
                                      ContentType="application/json",
                                      Body=json.dumps(Payloads[i]),
                                     )
    print(response["Body"].read().decode())

[{"label":"LABEL_3","score":0.5238500833511353}]
[{"label":"LABEL_3","score":0.9861999750137329}]
[{"label":"LABEL_1","score":0.9365861415863037}]
[{"label":"LABEL_1","score":0.7964444756507874}]
[{"label":"LABEL_3","score":0.9765886068344116}]
[{"label":"LABEL_1","score":0.9878882765769958}]
[{"label":"LABEL_3","score":0.9759885668754578}]
[{"label":"LABEL_1","score":0.971071720123291}]
[{"label":"LABEL_1","score":0.6195178627967834}]
[{"label":"LABEL_4","score":0.9910324215888977}]
[{"label":"LABEL_3","score":0.9304075241088867}]
[{"label":"LABEL_3","score":0.7927928566932678}]
[{"label":"LABEL_3","score":0.9872843027114868}]
[{"label":"LABEL_0","score":0.7753292918205261}]
[{"label":"LABEL_4","score":0.9681625962257385}]
[{"label":"LABEL_3","score":0.9863579273223877}]
[{"label":"LABEL_3","score":0.527592122554779}]
[{"label":"LABEL_4","score":0.5489237308502197}]
[{"label":"LABEL_3","score":0.9933062195777893}]
[{"label":"LABEL_3","score":0.985008955001831}]
[{"label":"LABEL_0","sc

## Clean up

Endpoints should be deleted when no longer in use, since (per the [SageMaker pricing page](https://aws.amazon.com/sagemaker/pricing/)) they're billed by time deployed.

In [ ]:
predictor.delete_endpoint()

## Conclusion

In this notebook you successfully ran SageMaker Training Job with the HuggingFace framework to fine-tune a pre-trained transformer on text classification using the `20 newsgroups dataset` dataset.
Then, you prepared the Serverless configuration required, and deployed your model to SageMaker Serverless Endpoint. Finally, you invoked the Serverless endpoint with sample data and got the prediction results.

As next steps, you can try running SageMaker Training Jobs with your own algorithm and your own data, and deploy the model to SageMaker Serverless Endpoint.